In [ ]:
import torch
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy.linalg import inv
from numpy.random import multivariate_normal
from notebook.services.config import ConfigManager
from traitlets.config.manager import BaseJSONConfigManager
import matplotlib as mpl
import math
from scipy.integrate import odeint, solve_ivp
import random
import torch.nn as nn
from typing import Callable, List

# import helper code
from utils import set_plot_attributes, interactive_projectile, interactive_pendulum, displacement_deriv, COLORS


# set jupyter configurations
%matplotlib inline
%config InlineBackend.figure_format='retina'
default_dpi = mpl.rcParamsDefault['figure.dpi']
mpl.rcParams['figure.dpi'] = default_dpi*1.2  


### 1.4.1 Ballistic motion problem
- projectile is launched with given launch velocity, launch angle, and a drag coefficient
- we can measure height of projectile with certain imprecision

In [ ]:
def projectile_sim(θ:np.array)->dict:
    """ Blackbox simulator that takes in a parameter vector and outputs a synthetic observation. """
    launch_velocity, launch_angle, drag_coefficient = θ
    drag_constant = 0.5 * drag_coefficient * 1.28 * 0.008 # 0.5 * drag_coefficent * air_density * projectile_area
    deriv = lambda t,u: (u[1],
                         -drag_constant/0.2 * np.hypot(u[1],u[3]) * u[1],
                         u[3],
                         -drag_constant/0.2 * np.hypot(u[1], u[3]) * u[3] - 9.81)
    
    # Initial conditions: x0, v0_x, z0, v0_z.
    u0 = 0, launch_velocity * np.cos(np.radians(launch_angle)), 0., launch_velocity * np.sin(np.radians(launch_angle))
    # Set range where we want to integrate
    t0, tf = 0, 400
    solution = solve_ivp(deriv, (t0, tf), u0, dense_output=True) # df/dt = f(t,y); f(t_i) = y_i
    # A fine grid of time points from 0 until impact time.
    time = np.linspace(0, tf, 2000)
    # get solution of ivp -> distance and height at each time point
    solution = solution.sol(time)
    distance_travelled, height = solution[0], solution[2]
    
    measurements = height + np.random.randn(distance_travelled.shape[0]) * 3
    
    return dict(θ=θ, distance_travelled=distance_travelled, height=height, x=measurements)


ballistic_motion = interactive_projectile(projectile_sim)
ballistic_motion

### 1.4.2 Linear regression as baseline
- how can we find the underlying parameters of the observation?
- with linear regression on quadratic polynomial parameters? 

In [ ]:
# get parameter and observation
θ = np.array(ballistic_motion.result['θ'])
x_o = ballistic_motion.result['x']
domain = ballistic_motion.result['distance_travelled']
# linear regression baseline
features = np.stack([domain**0, domain**1, domain**2]).T
linreg_param = inv(features.T @ features).dot(features.T @ x_o)
linreg_reconstruction = features @ linreg_param

In [ ]:
# plotting
fig, ax = plt.subplots()
ax.scatter(domain, x_o, marker='x', label='$x_o$', color=COLORS['data'], alpha=0.3)
ax.plot(domain, linreg_reconstruction, label = 'Linear regression prediction', color=COLORS['linreg'])
set_plot_attributes(ax, legend=True, ylim=(0, 250), xlabel='distance (m)', ylabel='height (m)')

### 1.4.3 Try to reproduce data manually 🛠️
- assuming you can build a forward model
- can you beat the linear regression model?

In [ ]:
def distance(prediction:np.array, data:np.array, distance_function:Callable) -> float:
    """ Second-order function that takes in a distance function and its arguments and returns the result. """
    return distance_function(prediction, data)


# sample parameters from a uniform distribution
θ = np.array([np.random.uniform(50, 250), np.random.uniform(10, 45), np.random.uniform(0.2, 0.7)])

# pass parameters to simulator and see result
simulation = projectile_sim(θ)

x_reconstruction = simulation['x']
domain = simulation['distance_travelled']

# θ
# plotting
fig, ax = plt.subplots()
ax.plot(domain,x_reconstruction, label='x_reconstruction', color=COLORS['abc'])
ax.scatter(domain, x_o, marker='x', label='x_o', color=COLORS['data'], alpha=0.3)

# choose a distance function
l2 = lambda prediction, data: np.square(prediction - data).mean()
chebyshev = lambda prediction, data: max([np.abs(data[i]-prediction[i]) for i in range(data.shape[0])])

# and an acceptance threshold ε
ε = 100000

# assesed visually, what would be a good acceptance threshold?
set_plot_attributes(ax, legend=True, ylim=(0,150), xlabel='distance (m)', ylabel='height (m)')


print(f'Distance of simulation with parameters launch velocity={θ[0]:.1f},\
                    launch angle={θ[1]:.1f},\
                    and drag coefficient={θ[2]:.2f}: \033[1m{distance(x_reconstruction, x_o, l2):.2f}\033[0m')

print(f'Based on chosen \033[1mε={ε}\033[0m, the simulation was \033[1m\
{"accepted" if (distance(x_reconstruction, x_o, l2) <= ε) else "not accepted"}\033[0m')

### 1.4.4 Automate the search for parameters 🛠️
- there should be a smarter way to approach this
- your task is now to automate what we just tried manually and make decisions which paramters to keep, based on distance

In [ ]:
def rejection_abc(ε:float, distance_function:Callable, num_simulations:int=1000)-> List:
    """ Can your approach be automated? """
    θ_accepted = []
    
    for _ in range(num_simulations):
        # sample θ from a uniform prior
        θ = np.array([np.random.uniform(50, 250), np.random.uniform(10, 45), np.random.uniform(0.2, 0.7)])
        ######################################################################## Solution
        # pass to simulator
        simulation = projectile_sim(θ)
        domain, x_reconstruction = simulation['distance_travelled'], simulation['x']
        # compute distance
        dist = distance(x_o, x_reconstruction, distance_function)
        # check if within epsilon
        if dist <= ε:
            # if so return parameters
            θ_accepted.append(θ)
        ######################################################################## Solution end
    return θ_accepted

ε = 1000
θ_accepted = rejection_abc(ε, l2)
print(f'Your rejection-ABC implementation has found {len(θ_accepted)} parameters that produce observations within ε={ε}.')

In [ ]:
# plot with one of the found parameters
θ = random.choice(θ_accepted)
sim_result = projectile_sim(θ)

# plotting
fig, ax = plt.subplots()
ax.plot(sim_result['distance_travelled'], sim_result['x'], label='Reconstruction', color=COLORS['abc'])
ax.scatter(sim_result['distance_travelled'], x_o, marker='x', label='Real measurements', color=COLORS['data'], alpha=0.3)
# plotting_boilerplate(ax, ylim=0, legend=True)
set_plot_attributes(ax, legend=True, ylim=(0,150), xlabel='distance (m)', ylabel='height (m)')

print(f'Distance of simulation with automatically found parameters launch velocity={θ[0]:.1f},\
                    launch angle={θ[1]:.1f}, and drag coefficient={θ[2]:.2f}: \033[1m{distance(sim_result["x"], x_o, l2):.2f}\033[0m')

- You just discovered rejection ABC!
- Reflect on how choice of $\epsilon$, $d$ was made

### 1.4.5 When does this break?
- suppose our observation is more complex

In [ ]:
def pendulum_sim(θ:np.array)->np.array:
    """
    Blackbox simulator that takes in a parameter vector
    and produces an observation of angular dispalement
    of a damped pendulum.
    """
    dampening_factor, mass, length = θ
    theta_0 = [0,3]
    t = np.linspace(0,20,150)    
    # solve ODE
    solution = odeint(displacement_deriv, theta_0,t,args =(dampening_factor,mass,9.81,length))
    angular_velocity_measurements =  solution[:,0] + np.random.randn(t.shape[0]) * 0.05
    # return observation
    return angular_velocity_measurements

pendulum = interactive_pendulum(pendulum_sim)
pendulum

### 1.4.6 What can we do about this?
- What summary statistics for this data would come to your  mind?
- Can we learn summary statistics with neural networks?

### 1.4.7 Introduce SBI as a potentially useful approach to solve our problem

In [ ]:
# define an initial, three-dimensional prior 
num_dim = 3
prior = utils.BoxUniform(low=torch.Tensor([0,0.01,1]), high=torch.Tensor([1,5,4]))

# and infer the posterior over θ using the sbi toolkit
posterior = infer(pendulum_sim, prior, method='SNPE', num_simulations=500)

In [ ]:
x_o = pendulum.result['x_o']
samples = posterior.sample((10000,), x=x_o)
θ = samples.mean(dim=0).numpy()
log_probability = posterior.log_prob(samples, x=x_o)
_ = analysis.pairplot(samples, figsize=(6,6))

In [ ]:
_, ax = plt.subplots()
ax.plot(np.linspace(0,20,150), pendulum_sim(θ), label='sbi reconstruction', color=COLORS['sbi'])
ax.plot(np.linspace(0,20,150), x_o, label='x_o', color=COLORS['data'], marker='x')
set_plot_attributes(ax, legend=True, xlabel='$t$', grid=True, ylabel='Angular displacement')